# Live System FULL CODE with Forecast

In [3]:
#################################### Step 1 Reading Data from CRM (via REST API) ########################################

import requests # library to perform HTTP requests
import pandas as pd # for tables
from pandas.io.json import json_normalize
import re

token = 'your API Key'
url = 'https://your-domain.freshsales.io'


# Retrieve deal stages
link = url + '/api/selector/deal_stages'                           
deal_stages = requests.get(link, \
                           headers={'Authorization': 'Token token='+token, \
                                 'Content-Type': 'application/json'}).json()
deal_stages =  json_normalize(deal_stages, 'deal_stages')[['id', 'name']]
deal_stages.rename(columns = {'id': 'deal_stage_id', 'name' : 'opportunity'}, inplace = True)

# Retrieve owners
link = url + '/api/selector/owners'                           
owners = requests.get(link, \
                           headers={'Authorization': 'Token token='+token, \
                                 'Content-Type': 'application/json'}).json()
owners = json_normalize(owners, 'users')[['id', 'display_name']]
owners.rename(columns = {'id' : 'owner_id', 'display_name' : 'owner'}, inplace = True)

# Retrieve territories
link = url + '/api/selector/territories'             
territories = requests.get(link, \
                         headers={'Authorization': 'Token token='+token, \
                                 'Content-Type': 'application/json'}).json()
territories =  json_normalize(territories, 'territories')[['id', 'name']]
territories.rename(columns = {'id' : 'territory_id', 'name': 'region'}, inplace = True)


# Retrieve total_pages
link = url + '/deals/view/3000791874'
total_pages = requests.get(link, \
                           headers={'Authorization': 'Token token='+token, \
                                 'Content-Type': 'application/json'}).json()['meta']['total_pages']


# Retrieve Deals 
i = 1
deals = pd.DataFrame()

     # Pagination
while i <= total_pages:
    link = url + '/deals/view/3000791874?page='+str(i)+'&include=deal_stage,territory,owner'   
    response = requests.get(link, \
                                headers={'Authorization': 'Token token='+token, \
                                         'Content-Type': 'application/json'})
        
    i_deals = json_normalize(response.json(), 'deals')  
    deals = pd.concat([deals, i_deals], axis = 0) # Append new response to former response dataframe
    i += 1

deals.rename(columns = {'id' : 'deal_id', 'name': 'deal_name'}, inplace = True)
deals.reset_index(inplace = True, drop = True) 

# Resolve dictionaries in columns custom_field and collaboration
df_custom_fields = deals['custom_field'].apply(pd.Series)
df_collaboration = deals['collaboration'].apply(pd.Series) 

# Concatenate along columns (axis = 1) and drop old unresolved column
deals = pd.concat([deals, df_custom_fields], axis = 1).drop('custom_field', axis=1)
deals = pd.concat([deals, df_collaboration], axis = 1).drop('collaboration', axis=1)
# Renaming Deal ID and Deal name
deals.rename(columns = {'name_x': 'deal_name', 'id_x': 'deal_id'}, inplace = True)


############################## Step 2 Merge and Transformations #################################

# Merge deals with Deal stages, Territories, owners  again
deals = pd.merge(deals, deal_stages, how = 'left',  left_on = 'deal_stage_id', right_on = 'deal_stage_id')
deals = pd.merge(deals, territories, how = 'left', left_on = 'territory_id', right_on = 'territory_id')
deals = pd.merge(deals, owners, how = 'left', left_on = 'owner_id', right_on = 'owner_id')

# Renaming some Columns
deals = deals.rename(columns={'amount': 'total_amount',
                              'cf_service_s': 'service', 'cf_product_s': 'solution', \
                              'cf_estimated_project_length' : 'estimated_project_length_month', \
                              'base_currency_amount' : 'order entry volume_USD' })

# DataType transformations
deals['total_amount'] = deals['total_amount'].astype('float').map(lambda x: '{:.2f}'.format(x)).astype('float')
deals['order entry volume_USD'] = deals['order entry volume_USD'].astype('float').map(lambda x: '{:.2f}'.format(x)).astype('float')

# Enable custom deal_stage sort in PowerBI
dict = {'New Opportunity (<25%)' : 1, 'Bridge 1 (25-50%)': 2, 'Bridge 2 (51-75%)': 3, \
        'Must Win (76-98%)': 4, 'Sign & Bill (99%)': 5, 'Won': 6, 'Lost' : 7 }
deals['custom_deal_id'] = deals.opportunity.map(dict)


############################## Step 3 Transformations for Financial Forecast #################################

# Transformation for Forecaast
deals.fillna(value=pd.np.nan, inplace=True)
deals['start_date'] = pd.to_datetime(deals['expected_close'])
deals['start_date'] = pd.to_datetime(deals['start_date'].dt.strftime('%Y/%m/%d')) # still have to fix date conversion
deals.drop('expected_close', axis = 1)
deals.estimated_project_length_month = deals.estimated_project_length_month.fillna(0)
deals.loc[:, 'estimated_project_length_month'] = deals.estimated_project_length_month.apply(lambda x: re.sub('months', '', str(x)))
deals.loc[:, 'estimated_project_length_month'] = deals.estimated_project_length_month.apply(lambda x: re.sub('month', '', str(x)))
deals.loc[:, 'estimated_project_length_month'] = deals.estimated_project_length_month.astype('int')

# Create two DataFrames out of API Result (=deals)
# Df_false - contains rows where a calculation Forecast is not possible
# df_true - Forecast possible
df_false = deals.loc[deals['estimated_project_length_month'] == 0, :].reset_index().drop(columns = 'index', axis = 1)
df_true = deals.loc[deals['estimated_project_length_month'] != 0, :].reset_index().drop(columns = 'index', axis = 1)
df_true['avg_amount'] = df_true['order entry volume_USD']
df_true['period'] = df_true['start_date']


###################### Step 4 Linear Forecast (dynamic row generation based on projet lenght) ##############

# Create, Transform and Perform linear Forecast of deal value(amount)
df_true2 = pd.DataFrame(columns = df_true.columns)

for index, row in df_true.iterrows():

       start_date = df_true.iloc[index]['start_date']
       duration = df_true.iloc[index]['estimated_project_length_month']
       amount = df_true.iloc[index]['order entry volume_USD']
       avg_amount = amount/duration

       for i in range(duration):
           start_date = start_date + pd.DateOffset(months=1)
           row['period'] = start_date
           row['avg_amount'] = avg_amount
           df_true2 = df_true2.append([[row][0]])


############################## Step 5 Concatenate df_false and df_true again #################################

df_true2['period2'] = pd.DatetimeIndex(df_true2['period']).to_period('M')

# Concatenate both DataFrames  again
FinalDF = pd.concat([df_false, df_true2], axis = 0, sort=True)

#FinalDF.drop(['level_0', 'index'], axis = 1, inplace = True)
FinalDF['avg_amount'] = FinalDF['avg_amount'].fillna(0).astype('int')


############################## Step 6 Filtering for Power BI #################################
# Add Columns to this DataFrame-Slicing if you need more columns in Power BI showing up

FinalDF = FinalDF[['deal_id', 'deal_name', 'opportunity', 'total_amount', 'order entry volume_USD', \
                   'estimated_project_length_month', 'start_date', 'period', 'avg_amount', \
                   'cf_movilitas_domain', 'region', 'service', 'solution', 'expected_close', \
                  'cf_sub-domain_we_track', 'cf_sub-domain_we_mobilize', 'cf_monthly_license_value', \
                  'cf_license_length_in_months', 'cf_project_location_country', \
                  'owner', 'custom_deal_id']]

FinalDF.sort_values(by = ['deal_name'])
FinalDF.reset_index(inplace = True)
print('Execution successful')

Execution successful


In [4]:
print("df_true : ", df_true.shape)
print("df_false : ", df_false.shape)

print("FinalDF: " , FinalDF.shape)
print(FinalDF.deal_name.nunique())
FinalDF.opportunity.unique()

df_true :  (22, 107)
df_false :  (261, 105)
FinalDF:  (386, 22)
282


array(['New Opportunity (<25%)', 'Bridge 2 (51-75%)', 'Bridge 1 (25-50%)',
       'Must Win (76-98%)', 'Sign & Bill (99%)'], dtype=object)